In [1]:
import numpy as np
import torch
import Doc_import
import os
import json
import Data_Preprocessing

In [2]:
from transformers import BertModel

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import Data_Preprocessing

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        kernel = 4
        input_ff = int(393216/(kernel * kernel))
        zwischenlayer1 = int(1000)
        zwischenlayer2 = int(10)
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.pooling = nn.MaxPool2d(kernel)
        self.linear11 = nn.Linear(input_ff, zwischenlayer1)
        self.linear12 = nn.Linear(zwischenlayer1, zwischenlayer2)
        self.linear13 = nn.Linear(zwischenlayer2, 1)

    def forward(self, input_data):
        bert_output = self.bert(**input_data)
        bert_output = self.pooling(bert_output[0])
        bert_output = bert_output[0].view(-1, 24576)
        x1 = F.relu(self.linear11(bert_output))
        x1 = F.relu(self.linear12(x1))
        x1 = self.linear13(x1)

        return x1

    def train_x1(self, features, labels, loss_function, optimizer, epochs):
        log_interval = 5
        for epoch in range(epochs):
            for i, data in enumerate(train_features):
                data = Data_Preprocessing.preprocessing(data)
                optimizer.zero_grad()
                output1 = self.forward(data)

                loss_x1 = loss_function(output1, torch.tensor(labels[i]['multi-author']))
                print(' ist=', output1, ' soll=', torch.tensor(labels[i]['multi-author']), ' Loss=', loss_x1)
                loss_x1.backward(retain_graph=True)
                optimizer.step()
                #if index % log_interval == 0:
                 #   print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epochs, index * len(features),
                  #                                                                 len(features),
                   #                                                                100. * index / len(features),
                    #                                                               loss_x1.data.item()))

    def test_model(self, features, labels, loss_function):
        log_interval = 5
        for index, data in enumerate(features):
            output = self(data)
            loss = loss_function(output, labels[index]['multi-author'])

            if index % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(index, index * len(data),
                                                                               len(data),
                                                                               100. * index / len(data),
                                                                               loss.data.item()))


In [3]:
train_features, train_labels, test_features, test_labels = Doc_import.import_pan21(base_path=r"C:\Users\jan-p\Documents\Uni Wuppertal\Semester 2\Information "
                                 r"Retrival\Projekt\Data\pan21")


In [4]:
#for data in train_features[0:3]:
 #   print(data ,'\n')

In [5]:
model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
criterion = nn.L1Loss()
model.train()
model.train_x1(train_features,train_labels , criterion, optimizer, 1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\jan-p\anaconda3\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input si

 ist= tensor([[-0.3892]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(1.3892, grad_fn=<L1LossBackward0>)
 ist= tensor([[-0.2074]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(1.2074, grad_fn=<L1LossBackward0>)
 ist= tensor([[-0.0363]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(1.0363, grad_fn=<L1LossBackward0>)
 ist= tensor([[0.5139]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(0.4861, grad_fn=<L1LossBackward0>)
 ist= tensor([[1.8517]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(0.8517, grad_fn=<L1LossBackward0>)
 ist= tensor([[1.7176]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(0.7176, grad_fn=<L1LossBackward0>)
 ist= tensor([[0.2642]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(0.7358, grad_fn=<L1LossBackward0>)
 ist= tensor([[-0.0399]], grad_fn=<AddmmBackward0>)  soll= tensor(1)  Loss= tensor(1.0399, grad_fn=<L1LossBackward0>)
 ist= tensor([[-0.1199]], grad_fn=<AddmmBackward0>)  soll= t

KeyboardInterrupt: 